**Data Pipeline:**

# New Section

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Setup working directory
%cd /content/drive/My Drive/Colab Notebooks/FP2

/content/drive/My Drive/Colab Notebooks/FP2


In [ ]:
# Create requirements file
%%writefile data_pipeline_requirements.txt

yfinance>=0.1.55
numpy>=1.12.0
pandas>=0.19
yahoo_fin
psycopg2

Overwriting data_pipeline_requirements.txt


In [ ]:
 !pip install -r data_pipeline_requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 9.1 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=e39e461ee2200439ce5267781ce4556652c0c9fab7f33e0f6504bcc2e81ef314
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6048 sha256=dde44e9170ec4e12fc1537d85496a55ca7da0b8669e9735a3dcacf41d964e0ab
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built bs4 sgmllib3k
  Attempting uninstall: urllib3
    Found existi

In [ ]:
# Import necessary libraries

import numpy as np
import pandas as pd
from yahoo_fin.stock_info import get_data
import datetime
from datetime import datetime
# import yfinance as yf
import psycopg2
from sqlalchemy import create_engine

In [ ]:
# Setup database connection details
db_credentials = {
    'user': 'goutham',
    'password': 'SharpMinded',
    'host': 'my-term5-db-instance.cdcogaqzqvwg.us-east-1.rds.amazonaws.com',
    'database': 'term5_default_database'
}

# Connection using psycopg2
conn = psycopg2.connect(**db_credentials)

In [ ]:
# Define the stock ticker and date range
Stock_ticker = "AAPL"
start_date = "2008-04-01"
end_date = datetime.now().strftime('%Y-%m-%d')  # Formats the current date as a string

In [ ]:
# Daily data
daily_data=get_data(Stock_ticker, start_date=start_date, end_date=end_date, index_as_date = False, interval="1d")
daily_data = pd.DataFrame(daily_data)

# Monthly data
monthly_data=get_data(Stock_ticker, start_date=start_date, end_date=end_date, index_as_date = False, interval="1mo")
monthly_data = pd.DataFrame(monthly_data)

print("daily rows imported: ", len(daily_data))
print("monthly rows imported: ", len(monthly_data))
daily_data.tail()

daily rows imported:  3950
monthly rows imported:  190


,date,open,high,low,close,adjclose,volume,ticker
3945,2023-11-30,189.839996,190.320007,188.190002,189.949997,189.949997,48794400,AAPL
3946,2023-12-01,190.330002,191.559998,189.229996,191.240005,191.240005,45679300,AAPL
3947,2023-12-04,189.979996,190.050003,187.449997,189.429993,189.429993,43389500,AAPL
3948,2023-12-05,190.210007,194.399994,190.179993,193.419998,193.419998,66628400,AAPL
3949,2023-12-06,194.449997,194.759995,192.110001,192.320007,192.320007,41060000,AAPL


In [ ]:
# Drop NA

daily_data.dropna(how='any',inplace=True)
print("daily rows after dropping na: ", len(daily_data))
monthly_data.dropna(how='any',inplace=True)
print("monthly rows after dropping na: ", len(monthly_data))

daily rows after dropping na:  3950
monthly rows after dropping na:  190


In [ ]:
# Using pandas to_sql to insert data into the PostgreSQL table
engine = create_engine(f'postgresql+psycopg2://{db_credentials["user"]}:{db_credentials["password"]}@{db_credentials["host"]}/{db_credentials["database"]}')

daily_table_name = 'daily_data_aapl'
daily_data.to_sql(daily_table_name, engine, if_exists='replace', index=False)

monthly_table_name = 'monthly_data_aapl'
monthly_data.to_sql(monthly_table_name, engine, if_exists='replace', index=False)

190

In [ ]:
#Check db to confirm table writes

daily_table_name = 'daily_data_aapl'
monthly_table_name = 'monthly_data_aapl'

# Query to select all rows from the table
daily_select_query = f"SELECT * FROM {daily_table_name}"
monthly_select_query = f"SELECT * FROM {daily_table_name}"

# Using pandas read_sql to read data into a DataFrame
daily_df = pd.read_sql(daily_select_query, conn)
monthly_df = pd.read_sql(monthly_select_query, conn)

# Closing the connection
conn.close()

<ipython-input-13-4e6e6fcaa8b6>:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  daily_df = pd.read_sql(daily_select_query, conn)
<ipython-input-13-4e6e6fcaa8b6>:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  monthly_df = pd.read_sql(monthly_select_query, conn)


In [ ]:
print("daily rows after dropping na: ", len(daily_data))
print("monthly rows after dropping na: ", len(monthly_data))

daily rows after dropping na:  3950
monthly rows after dropping na:  190
